Importing libraries

In [1]:
import numpy as np
from bs4 import BeautifulSoup #for data scraping
import requests #for bs4
import re #for regular expressions
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords

Downloading data for nltk

In [2]:
nltk.download('stopwords') #downloading nltk stopwords
nltk.download('punkt') #for sent_tokenize and word_tokenize

[nltk_data] Downloading package stopwords to /home/nitin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/nitin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Read excel data

In [3]:
df = pd.read_excel('cik_list.xlsx')
df.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   CIK       152 non-null    int64         
 1   CONAME    152 non-null    object        
 2   FYRMO     152 non-null    int64         
 3   FDATE     152 non-null    datetime64[ns]
 4   FORM      152 non-null    object        
 5   SECFNAME  152 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 7.2+ KB


# Preprocessing

In [5]:
df.isnull().sum()

CIK         0
CONAME      0
FYRMO       0
FDATE       0
FORM        0
SECFNAME    0
dtype: int64

No null values found.

Next, completing urls by adding "https://www.sec.gov/Archives/" to in front each value of "SECFNAME" column.

In [6]:
starting_url = "https://www.sec.gov/Archives/"
df_orig = df.copy(deep=True) #creating copy of original dataframe for later use
df['SECFNAME'] = starting_url + df['SECFNAME']

Generating data lists (using dataframes) from excel files

In [7]:
dict_constraining = pd.read_excel('constraining_dictionary.xlsx') #header = word
dict_constraining = dict_constraining['Word'].tolist() #converting dataframe to list
dict_constraining = [word.lower() for word in dict_constraining] #converting to lowercase

dict_uncertainty = pd.read_excel('uncertainty_dictionary.xlsx') #header = word
dict_uncertainty = dict_uncertainty['Word'].tolist() #converting dataframe to list
dict_uncertainty = [word.lower() for word in dict_uncertainty] #converting to lowercase

dict_stopwords_generic = pd.read_csv('extra/StopWords_Generic.txt', header=None) #no headers in this text file
dict_stopwords_generic = dict_stopwords_generic.iloc[:,0].tolist()

In [8]:
#next is master dictionary which contains positive and negative words
filepath = 'extra/LoughranMcDonald_SentimentWordLists_2018.xlsx'

#for postive word list
dict_sentiment_word_lists_positive = pd.read_excel(filepath, sheet_name = 'Positive', header=None)
dict_sentiment_word_lists_positive = dict_sentiment_word_lists_positive[0].tolist() #converting dataframe to list
dict_sentiment_word_lists_positive = [word.lower() for word in dict_sentiment_word_lists_positive] #converting to lowercase

#for negative word list
dict_sentiment_word_lists_negative = pd.read_excel(filepath, sheet_name = 'Negative', header=None)
dict_sentiment_word_lists_negative = dict_sentiment_word_lists_negative[0].tolist() #converting dataframe to list
dict_sentiment_word_lists_negative = [word.lower() for word in dict_sentiment_word_lists_negative] #converting to lowercase

# Functions to be used for extraction, cleaning and analysis

Extraction:

In [9]:
def extract_data_bs4(url):
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "lxml") #requires lxml package
    return soup.prettify()

Clean html data:

In [10]:
def clean_data(data):
    cleaned_data = re.sub('[^A-Za-z \.\'\?]+','',data) #removing digits, special characters except "? '."
    cleaned_data = re.sub(' +',' ',cleaned_data) #replacing one or more whitespaces with a single space
    cleaned_data = re.sub(' \.','.',cleaned_data) #replacing " ." with "."
    cleaned_data = re.sub('\.+','.',cleaned_data) #replacing multiple "." with a single "."
    return cleaned_data

Remove stopwords:

In [11]:
def remove_stopwords(stop_words,cleaned_word_tokens):
    removed_list_stopwords = [word for word in cleaned_word_tokens if not word in stop_words] 
    return removed_list_stopwords

Count syllables:

In [12]:
def count_syllable(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

Check if word is complex:

In [13]:
def check_complex(word):
    count = count_syllable(word)
    if count > 2:
        return True
    return False

# Functions to be calculated

Positive Score:

In [14]:
def calculate_positive_score(cleaned_data_words):
    count = 0
    for word in cleaned_data_words:
        if word in dict_sentiment_word_lists_positive:
            count = count + 1
    return count

Negative Score:

In [15]:
def calculate_negative_score(cleaned_data_words):
    count = 0
    for word in cleaned_data_words:
        if word in dict_sentiment_word_lists_negative:
            count = count + 1
    return count

Polarity score:

In [16]:
def calculate_polarity_score(positive_score, negative_score):
    return (positive_score - negative_score)/((positive_score + negative_score) + 0.000001)

Average sentence length:

In [17]:
def sentence_avg_len(no_of_words, no_of_sentences):
    return no_of_words/no_of_sentences

Percentage of complex words:

In [18]:
def percent_complex_words(no_complex_words, no_of_words):
    return no_complex_words/no_of_words

Fog index:

In [19]:
def fog_index(avg_sentence_length, percent_complex):
    return 0.4*(avg_sentence_length + percent_complex)

Uncertainty score:

In [20]:
def calculate_uncertainty_score(cleaned_data_words):
    count = 0
    for word in cleaned_data_words:
        if word in dict_uncertainty:
            count = count + 1
    return count

Constraining score:

In [21]:
def calculate_constraining_score(cleaned_data_words):
    count = 0
    for word in cleaned_data_words:
        if word in dict_constraining:
            count = count + 1
    return count

Constraining words for the whole report (only calculated once) for each row:

In [22]:
def constraining_words_whole_report(data):
    cleaned_data = clean_data(data)
    cleaned_data = re.sub('[\.\?]','',cleaned_data) #removing '.' and '?' for words
    word_tokens = word_tokenize(cleaned_data) #tokenize words 
    cleaned_data_words = [word for word in word_tokens if word] #remove empty string values
    cleaned_data_words = [word.lower() for word in cleaned_data_words] #converting all words into lowercase
    
    count = 0
    for word in cleaned_data_words:
        if word in dict_constraining:
            count = count + 1
    return count

# MAIN PART

14 variables to be calculated for 3 sections:

In [23]:
section_names = ['mda','qqdmr','rf']
header_columns = ['positive_score','negative_score','polarity_score','average_sentence_length', 'percentage_of_complex_words','fog_index','complex_word_count','word_count','uncertainty_score','constraining_score','positive_word_proportion','negative_word_proportion','uncertainty_word_proportion','constraining_word_proportion']
section_full_names = ['MANAGEMENT\'S DISCUSSION AND ANALYSIS','QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK','RISK FACTORS']

header_columns_with_sections = []
# all combinations
for name in section_names:
    for column in header_columns:
        header_columns_with_sections.append(name + '_' + column)

# finally appending last column
header_columns_with_sections.append('constraining_words_whole_report')

Creating new dataframe to store final values

In [24]:
data_dictionary = pd.concat([df_orig,pd.DataFrame(columns=header_columns_with_sections)])
data_dictionary = data_dictionary.fillna(0) #replacing all NaN values with zeroes
data_dictionary.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,...,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,3662.0,SUNBEAM CORP/FL/,199803.0,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3662.0,SUNBEAM CORP/FL/,199805.0,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3662.0,SUNBEAM CORP/FL/,199808.0,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3662.0,SUNBEAM CORP/FL/,199811.0,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3662.0,SUNBEAM CORP/FL/,199811.0,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# main loop

#NOTE: there is python error when parsing 64th and 65th url, so corresponding loops are skipped. see "error_63&64index.png"
    
#looping though each row
for row_index in range(0,len(df.index)):
    
    #skipping row with index 63 and 64
    if row_index==63 or row_index==64:
        continue #error occurs when parsing 64th and 65th row, see "error_63&64index.png"
    
    #extract data
    url = df['SECFNAME'][row_index]
    data = extract_data_bs4(url)
    
    #loop through 3 sections (mda,qqdmr,rf)
    section_index = 0
    for section in section_full_names:
        section_index = section_full_names.index(section)

        #check if particular section (out of given 3) is present on webpage or not...
        #...by matching "table of contents" value of webpage
        #for example, "ITEM 6.    MANAGEMENT'S DISCUSSION AND ANALYSIS" will be checked with regex
        pattern = 'ITEM [0-9a-zA-Z]+\.([ ]{1,}|\t)'+section

        #check if regex pattern matches data from web page
        if re.search(pattern,data,flags=re.IGNORECASE):
            cleaned_data = clean_data(data) #removed all brackets and digits

            cleaned_sent_orig = sent_tokenize(cleaned_data) #generate sentences

            cleaned_data_sentences = []
            for sentence in cleaned_sent_orig:
                cleaned_data_sentences.append(re.sub('[\.\?]','',sentence)) #removing '.' and '?' from sentences

            cleaned_data = re.sub('[\.\?]','',cleaned_data) #removing '.' and '?' for words
            word_tokens = word_tokenize(cleaned_data) #tokenize words

            stop_words = set(stopwords.words('english')) #nltk stopwords in english

            cleaned_data_words = [word for word in word_tokens if word] #remove empty string values
            cleaned_data_words = [word.lower() for word in cleaned_data_words] #converting all words into lowercase
            cleaned_data_sentences = [sentence.lower() for sentence in cleaned_data_sentences] #converting all sentences into lowercase

            #removing stopwords
            cleaned_data_words = remove_stopwords(stop_words,cleaned_data_words)

            #TOTAL SENTENCE COUNT
            cleaned_data_sentences_length = len(cleaned_data_sentences) #total number of sentences
            #TOTAL WORD COUNT
            cleaned_data_words_length = len(cleaned_data_words) #total number of words


            #******NEXT PART IS FOR CALCULATION OF VARIOUS FUNCTIONS******

            #POSITIVE SCORE
            positive_score = calculate_positive_score(cleaned_data_words)

            #NEGATIVE SCORE
            negative_score = calculate_negative_score(cleaned_data_words)

            #POLARITY SCORE
            polarity_score = calculate_polarity_score(positive_score, negative_score)

            #AVERAGE SENTENCE LENGTH
            sentence_avg_length = sentence_avg_len(cleaned_data_words_length, cleaned_data_sentences_length)

            #COMPLEX WORD COUNT
            complex_count = 0
            for word in cleaned_data_words:
                if check_complex(word):
                    complex_count = complex_count + 1

            #PERCENTAGE OF COMPLEX WORDS
            percentage_complex_words = percent_complex_words(complex_count, cleaned_data_words_length)

            #FOG INDEX
            fog_index_value = fog_index(sentence_avg_length, percentage_complex_words)

            #UNCERTAINITY SCORE
            uncertainty_score = calculate_uncertainty_score(cleaned_data_words)

            #CONSTRAINING SCORE
            constraining_score = calculate_constraining_score(cleaned_data_words)

            #POSITIVE WORD PROPORTION
            positive_word_proportion = positive_score/cleaned_data_words_length

            #NEGATIVE WORD PROPORTION
            negative_word_proportion = negative_score/cleaned_data_words_length

            #UNCERTAINTY WORD PROPORTION
            uncertainty_word_proportion = uncertainty_score/cleaned_data_words_length

            #CONSTRAINING WORD PROPORTION
            constraining_word_proportion = constraining_score/cleaned_data_words_length

            #Adding values back to final dataframe
            data_dictionary.loc[data_dictionary.index[row_index],[section_names[section_index]+'_positive_score']] = positive_score
            data_dictionary.loc[data_dictionary.index[row_index],[section_names[section_index]+'_negative_score']] = negative_score
            data_dictionary.loc[data_dictionary.index[row_index],[section_names[section_index]+'_polarity_score']] = polarity_score
            data_dictionary.loc[data_dictionary.index[row_index],[section_names[section_index]+'_average_sentence_length']] = sentence_avg_length
            data_dictionary.loc[data_dictionary.index[row_index],[section_names[section_index]+'_percentage_of_complex_words']] = percentage_complex_words
            data_dictionary.loc[data_dictionary.index[row_index],[section_names[section_index]+'_fog_index']] = fog_index_value
            data_dictionary.loc[data_dictionary.index[row_index],[section_names[section_index]+'_complex_word_count']] = complex_count
            data_dictionary.loc[data_dictionary.index[row_index],[section_names[section_index]+'_word_count']] = cleaned_data_words_length
            data_dictionary.loc[data_dictionary.index[row_index],[section_names[section_index]+'_uncertainty_score']] = uncertainty_score
            data_dictionary.loc[data_dictionary.index[row_index],[section_names[section_index]+'_constraining_score']] = constraining_score
            data_dictionary.loc[data_dictionary.index[row_index],[section_names[section_index]+'_positive_word_proportion']] = positive_word_proportion
            data_dictionary.loc[data_dictionary.index[row_index],[section_names[section_index]+'_negative_word_proportion']] = negative_word_proportion
            data_dictionary.loc[data_dictionary.index[row_index],[section_names[section_index]+'_uncertainty_word_proportion']] = uncertainty_word_proportion
            data_dictionary.loc[data_dictionary.index[row_index],[section_names[section_index]+'_constraining_word_proportion']] = constraining_word_proportion
    #CONSTRAINING WORDS FOR WHOLE REPORT
    constraining_words_report = constraining_words_whole_report(data)
    data_dictionary.loc[data_dictionary.index[row_index],'constraining_words_whole_report'] = constraining_words_report

In [26]:
data_dictionary.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,...,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,3662.0,SUNBEAM CORP/FL/,199803.0,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,878,2463,-0.474409,18.116701,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,1275
1,3662.0,SUNBEAM CORP/FL/,199805.0,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,445,1212,-0.462885,19.159197,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,871
2,3662.0,SUNBEAM CORP/FL/,199808.0,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,0,0,0.000000,0.000000,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,5
3,3662.0,SUNBEAM CORP/FL/,199811.0,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,309,1174,-0.583277,15.017590,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,613
4,3662.0,SUNBEAM CORP/FL/,199811.0,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0,0,0.000000,0.000000,...,0.0,0,0,0,0,0.0,0.0,0.0,0.0,4


Saving the output to excel file

In [27]:
data_dictionary.to_excel('SOLUTION_OUTPUT.xlsx') #requires openpyxl package